In [2]:
from mylib.statistic_test import *

code_id = '0808 - Footprint Over Time'
loc = join(figpath, "Dsp", code_id)
mkdir(loc)

f = f_CellReg_dsp

def select_roi(sfp: np.ndarray):
    boundaries = []
    for i in range(sfp.shape[2]):
        # Find contours in the binary image
        image = np.uint8(sfp[:, :, i]*255)
        contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Get the longest contour (assuming it is the boundary of the closed shape)
        longest_contour = max(contours, key=len)

        longest_contour = np.concatenate([longest_contour, [[[longest_contour[0, 0, 0], longest_contour[0, 0, 1]]]] ], axis=0)
        # Convert the contour points to numpy array
        traced_boundary = np.array(longest_contour)
        boundaries.append(traced_boundary)
        
    
    return boundaries

def visualize_sfps(mouse: int, cell_index):
    idx = np.where(f['MiceID'] == mouse)[0][0]
    
    folder_dir = f['sfp_folder'][idx]
    cellreg_dir = f['cellreg_folder'][idx]
    
    with open(cellreg_dir, 'rb') as handle:
        index_map: np.ndarray = pickle.load(handle)
        index_map = index_map.astype(np.int64)
        
        if mouse != 10232:
            index_map = index_map[1:, :]
    
    print(index_map.shape)
    
    sfps = []
    # Iterate all files in the folder
    for file in os.listdir(folder_dir):
        if file.endswith(".mat"):
            # Load the mat file
            mat_path = os.path.join(folder_dir, file)
            with h5py.File(mat_path, 'r') as mat_file:
                sfp = np.asarray(mat_file['SFP'])
            
            sfps.append(sfp)

    assert len(sfps) == index_map.shape[0]      
    # Visualize
    fig, axes = plt.subplots(ncols=7, nrows=1, figsize=(18, 4))
    colors = sns.color_palette("Paired", len(cell_index))
    
    for i in tqdm(range(len(sfps))):
        ax = Clear_Axes(axes[i])
        boundaries = select_roi(sfps[i])
        n = sfps[i].shape[2]
        for j in range(n):
            ax.plot(boundaries[j][:, 0, 0], boundaries[j][:, 0, 1], color = 'gray', linewidth = 0.2)
        
        for j, k in enumerate(cell_index):
            ax.plot(boundaries[index_map[i, k]-1][:, 0, 0], boundaries[index_map[i, k]-1][:, 0, 1], color = colors[j], linewidth = 1)
            #ax.text(boundaries[index_map[i, k]-1][0, 0, 0], boundaries[index_map[i, k]-1][0, 0, 1], f'{k}')
    
        ax.set_aspect('equal')
        ax.axis([0, 160, 0, 250])
    
    plt.savefig(join(loc, f'SFP {mouse}.png'), dpi=600)
    plt.savefig(join(loc, f'SFP {mouse}.svg'), dpi=600)
    plt.show()
    
    
#visualize_sfps(10232, np.array([0, 1, 2, 3, 4, 5, 7, 11, 12, 13, 15]))
#visualize_sfps(10227, np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11]))
#visualize_sfps(10224, np.array([2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 15]))
#visualize_sfps(10212, np.array([0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 15]))
#visualize_sfps(10209, np.array([2, 4, 5, 7, 10, 12, 15, 16]))

with open(f_CellReg_dsp['cellreg_folder'][2], 'rb') as handle:
    index_map = pickle.load(handle)
    
    print(index_map[1:, 2])

        E:\Data\FinalResults\Dsp\0808 - Footprint Over Time is already existed!
[ 69   8  21  51  71  74 222]
